#### Importando Bibliotecas

In [1]:
import duckdb
import os

from dotenv import load_dotenv

#### Primeiros Passos com MotherDuck

##### Configurando Token

In [2]:
dotenv_path = "../config/.env"
load_dotenv(dotenv_path)

motherduck_token =  os.getenv("motherduck_token")

##### Inicializando conexão do DuckDB ao MotherDuck

In [3]:
con = duckdb.connect(f"md:?motherduck_token={motherduck_token}")

In [4]:
con.sql("SHOW DATABASES")

┌───────────────┐
│ database_name │
│    varchar    │
├───────────────┤
│ my_db         │
│ sample_data   │
│ teste         │
└───────────────┘

##### Lendo o CSV

In [5]:
# usando AUTO_DETECT=TRUE para inferir o schema automático
con.sql("SELECT * FROM read_csv('../data/input/contratos_202404.csv', AUTO_DETECT=TRUE) LIMIT 5")

┌────────────────┬─────────────────────┬──────────────────────┬───┬─────────────────┬────────────────┬─────────────┐
│ DataAssinatura │ DataTerminoOriginal │ DataTterminoAposAd…  │ … │ ValorAtualizado │ ValorEmpenhado │  ValorPago  │
│      date      │        date         │         date         │   │     double      │     double     │   double    │
├────────────────┼─────────────────────┼──────────────────────┼───┼─────────────────┼────────────────┼─────────────┤
│ 2017-12-14     │ 2018-12-14          │ 2024-12-14           │ … │      1369848.55 │     1211677.61 │  1192042.88 │
│ 2017-11-10     │ 2018-11-10          │ 2024-11-11           │ … │       228445.44 │      207802.58 │   207882.78 │
│ 2017-11-01     │ 2018-10-31          │ 2024-10-31           │ … │        216720.0 │       190174.0 │   190131.24 │
│ 2017-11-28     │ 2018-11-30          │ 2024-11-30           │ … │         94752.5 │       80642.03 │    80642.03 │
│ 2017-11-21     │ 2019-05-21          │ 2024-11-20           │ 

##### Inserindo o CSV em uma tabela temporária chamada "stg"

In [6]:
con.sql("CREATE OR REPLACE TEMP TABLE stg AS SELECT * FROM read_csv('../data/input/contratos_202404.csv', AUTO_DETECT=TRUE)")

In [7]:
con.sql("SELECT * FROM temp.stg")

┌────────────────┬─────────────────────┬──────────────────────┬───┬─────────────────┬────────────────┬─────────────┐
│ DataAssinatura │ DataTerminoOriginal │ DataTterminoAposAd…  │ … │ ValorAtualizado │ ValorEmpenhado │  ValorPago  │
│      date      │        date         │         date         │   │     double      │     double     │   double    │
├────────────────┼─────────────────────┼──────────────────────┼───┼─────────────────┼────────────────┼─────────────┤
│ 2017-12-14     │ 2018-12-14          │ 2024-12-14           │ … │      1369848.55 │     1211677.61 │  1192042.88 │
│ 2017-11-10     │ 2018-11-10          │ 2024-11-11           │ … │       228445.44 │      207802.58 │   207882.78 │
│ 2017-11-01     │ 2018-10-31          │ 2024-10-31           │ … │        216720.0 │       190174.0 │   190131.24 │
│ 2017-11-28     │ 2018-11-30          │ 2024-11-30           │ … │         94752.5 │       80642.03 │    80642.03 │
│ 2017-11-21     │ 2019-05-21          │ 2024-11-20           │ 

##### Salvando os dados no MotherDuck

In [8]:
# criando o banco de dados
con.sql("CREATE DATABASE data_processing")

In [9]:
# criando um schema
con.sql("CREATE SCHEMA data_processing.dbo")

In [10]:
# criando a tabela automaticamente e inserindo os dados
# assim como a stg, estamos inferindo o schema da tabela automaticamente
con.sql("CREATE TABLE data_processing.dbo.contratos AS SELECT * FROM temp.stg")

##### Contando as Linhas

In [11]:
con.sql("SELECT COUNT(*) FROM data_processing.dbo.contratos")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        20543 │
└──────────────┘

##### Inserindo mais dados

In [12]:
# inserindo a stg novamente
con.sql("INSERT INTO data_processing.dbo.contratos SELECT * FROM temp.stg")

In [13]:
# contando novamente
con.sql("SELECT COUNT(*) FROM data_processing.dbo.contratos")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        41086 │
└──────────────┘

##### Criando uma View

In [16]:
con.sql("CREATE VIEW data_processing.dbo.view_rescisao AS SELECT * FROM data_processing.dbo.contratos WHERE DataRescisao IS NOT NULL")

##### Selecionando a View

In [17]:
con.sql("SELECT * FROM data_processing.dbo.view_rescisao")

┌────────────────┬─────────────────────┬──────────────────────┬───┬─────────────────┬────────────────┬───────────┐
│ DataAssinatura │ DataTerminoOriginal │ DataTterminoAposAd…  │ … │ ValorAtualizado │ ValorEmpenhado │ ValorPago │
│      date      │        date         │         date         │   │     double      │     double     │  double   │
├────────────────┼─────────────────────┼──────────────────────┼───┼─────────────────┼────────────────┼───────────┤
│ 2023-08-25     │ 2024-09-29          │ 2024-04-02           │ … │         10800.0 │            0.0 │       0.0 │
│ 2023-10-11     │ 2024-10-09          │ 2024-10-09           │ … │         12223.0 │        12223.0 │   12223.0 │
│ 2023-08-25     │ 2024-09-29          │ 2024-04-02           │ … │         10800.0 │            0.0 │       0.0 │
│ 2023-10-11     │ 2024-10-09          │ 2024-10-09           │ … │         12223.0 │        12223.0 │   12223.0 │
├────────────────┴─────────────────────┴──────────────────────┴───┴─────────────

##### Usando CTE

In [20]:
con.sql("WITH cteTeste AS (SELECT * FROM data_processing.dbo.view_rescisao) SELECT * FROM cteTeste WHERE DataAssinatura = '2023-08-25'")

┌────────────────┬─────────────────────┬──────────────────────┬───┬─────────────────┬────────────────┬───────────┐
│ DataAssinatura │ DataTerminoOriginal │ DataTterminoAposAd…  │ … │ ValorAtualizado │ ValorEmpenhado │ ValorPago │
│      date      │        date         │         date         │   │     double      │     double     │  double   │
├────────────────┼─────────────────────┼──────────────────────┼───┼─────────────────┼────────────────┼───────────┤
│ 2023-08-25     │ 2024-09-29          │ 2024-04-02           │ … │         10800.0 │            0.0 │       0.0 │
│ 2023-08-25     │ 2024-09-29          │ 2024-04-02           │ … │         10800.0 │            0.0 │       0.0 │
├────────────────┴─────────────────────┴──────────────────────┴───┴─────────────────┴────────────────┴───────────┤
│ 2 rows                                                                                    23 columns (6 shown) │
└───────────────────────────────────────────────────────────────────────────────

#### Continua.....